In [9]:
import requests
import re
import pdfplumber
import pandas as pd
import os
import io
import asyncio
import aiohttp
from bs4 import BeautifulSoup

In [2]:
# get df with all links
year = 2024
mainurl = f'https://www.cmdachennai.gov.in/ApprovedLayout/{year}.htm'
df = pd.read_html(mainurl)[0]
df = df.iloc[:-1,:-1]


resp = requests.get(mainurl)
resp.raise_for_status()
soup = BeautifulSoup(resp.text, 'html.parser')
pdflist = soup.find_all('a')
pdflist = [i for i in pdflist if i.get('href').endswith('.pdf')]
df['bs4'] = pdflist

def extractpplfromurl(url):
    return url.split('/')[-1][:-4].replace('-','/')

df['url'] = df.bs4.apply(lambda x: 'https://www.cmdachennai.gov.in/' + x.get('href'))
df['extracted_ppl'] = df.url.apply(extractpplfromurl)
df['pdf_savepath'] = df.extracted_ppl.apply(lambda x: x.split('/')[-1] +'/'+ x.split('/')[0] + '.pdf' )


In [7]:
# verify layout numbers match
pd.DataFrame(df.iloc[:,1] == df.iloc[:,-2]).all()



0    True
dtype: bool

In [28]:
async def downlaod_and_extract_pdf(session, df_row, x_start_ratio=0.4):
    pdf_url, savepath = df_row['url'], 'data/' + df_row['pdf_savepath']
    try:
        all_text = []
        savedtodisk = False
        # Use aiohttp for async HTTP requests
        async with session.get(pdf_url, ssl=False) as response:
            response.raise_for_status()
            content = await response.read()
            
            # Read PDF
            pdf_file = io.BytesIO(content)
            # save the pdf file
            os.makedirs(os.path.dirname(os.path.abspath(savepath)), exist_ok=True)
            with open(savepath, 'wb') as fh:
                fh.write(content)
                savedtodisk = True
                print(f"Saved PDF to disk: {savepath}")

            with pdfplumber.open(pdf_file) as pdf:
                for page in pdf.pages:
                    width = page.width
                    height = page.height
                    
                    # Dynamically compute the crop box based on width
                    # Adjust the start ratio (0.4 or 0.5) as needed
                    crop = page.within_bbox((width * x_start_ratio, 0, width, height))
                    text = crop.extract_text()
                    if text:
                        all_text.append(text)
        return "\n".join(all_text), savedtodisk
    except Exception as e:
        return f"ERROR: {e}", False
    

def extract_multiple_patterns(text: str) -> dict[str|None]:
    """
    Extract multiple patterns from PDF text
    
    Returns:
        Dictionary with pattern matches
    """
    # Define patterns with descriptive keys
    patterns = {
        'total_no_plots': [
            r'NO\.?\s*OF\.?\s*PLOTS\s*[=:]\s*(\d+)',
            r'No\.?\s*of\s*[Pp]lots\s*[=:]\s*(\d+)',
            r'TOTAL\s*PLOTS?\s*[=:]\s*(\d+)'
        ],
        'ews': [
            r'(?i)\bE\.?W\.?S\.?\s+(?:Provided|Plots)\b[^\n]*[:=]\s*(\d+)\s*Nos\.?\b'
        ],
        'regular':[
            r'(?i)REGULAR PLOTS.\s*[^\n]*?[:=]\s*(\d+)\s*No'
        ],
        'date': [
            r'DATE\s*[=:]\s*(\d{1,2}\s?[-\/]\s?\d{2}\s?[-\/]\s?\d{4})',
            r'Date\s*:\s*(\d{1,2}(?:st|nd|rd|th)?\s+(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)\s+\d{4})',
            r'(\d{4}-\d{2}-\d{2})'
        ],
        'reference_number': [
            r'REF(?:ERENCE)?\s*NO\.?\s*[=:]\s*([A-Z0-9-/]+)',
            r'File\s*No\.?\s*[=:]\s*([A-Z0-9-/]+)',
            r'Application\s*No\.?\s*[=:]\s*([A-Z0-9-/]+)'
        ]
    }

    try:
        
        # Dictionary to store results
        results = {}
        
        # Search for each pattern category
        for category, pattern_list in patterns.items():
            results[category] = None
            for pattern in pattern_list:
                matches = re.findall(pattern, text, re.IGNORECASE)
                if matches:
                    results[category] = matches[0]
                    break
        
        return results
        
    except Exception as e:
        print(f"Error processing URL {text}: {str(e)}")
        return {key: None for key in patterns.keys()}, text

# Example usage with DataFrame
# extract_multiple_patterns(df['url'][2])

In [32]:
# Now populate the columns from the extracted patterns
def extract_and_expand(text):
    if text and not isinstance(text, str) or (isinstance(text, str) and not text.startswith("ERROR:")):
        patterns = extract_multiple_patterns(text)
        return patterns
    return {key: None for key in ['total_no_plots', 'ews', 'regular', 'date', 'reference_number']}

# Usage with asyncio
async def process_df(df):
    results_text = []
    results_saved = []
    async with aiohttp.ClientSession() as session:
        tasks = [downlaod_and_extract_pdf(session, row) for _, row in df.iterrows()]
        batch_results = await asyncio.gather(*tasks)
        
        # Unpack results
        batch_text, batch_saved = zip(*batch_results)
        results_text.extend(batch_text)
        results_saved.extend(batch_saved)
        
    result_df = df.copy()
    result_df['extracted_text'] = results_text
    result_df['saved_to_disk'] = results_saved
    return result_df


async def main():
    # Process first 20 rows as an example
    result_df = await process_df(df)
    
    # Now you can use the DataFrame as usual
    print(f"Successfully saved {result_df['saved_to_disk'].sum()} of {len(result_df)} PDFs")
    
    # Apply pattern extraction to the extracted text and expand into separate columns
    for key in ['total_no_plots', 'ews', 'regular', 'date', 'reference_number']:
        result_df[key] = None  # Initialize columns


    # Apply the extraction function
    pattern_dicts = result_df['extracted_text'].apply(extract_and_expand)

    # Expand the dictionaries into separate columns
    for key in ['total_no_plots', 'ews', 'regular', 'date', 'reference_number']:
        result_df[key] = pattern_dicts.apply(lambda x: x.get(key))
    
    return result_df

# Run the async function
result_df = await main()

CropBox missing from /Page, defaulting to MediaBox


Saved PDF to disk: data/2024/97.pdf


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


Saved PDF to disk: data/2024/75.pdf
Saved PDF to disk: data/2024/47.pdf


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


...

Saved PDF to disk: data/2024/181.pdf


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


Saved PDF to disk: data/2024/276.pdf


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


Saved PDF to disk: data/2024/178.pdf
Successfully saved 275 of 309 PDFs


In [35]:
pqdf = result_df.drop(columns='bs4')
pqdf.to_parquet('./2024extract.parquet')

In [37]:
pqdf[pqdf.extracted_text.str.startswith("ERROR")]

,SNo.,Approval No. PPD/L.O. No.,Localbody,Village,url,extracted_ppl,pdf_savepath,extracted_text,saved_to_disk,total_no_plots,ews,regular,date,reference_number
6,7,07/2024,Thiruverkadu MP,Noombal,https://www.cmdachennai.gov.in/pdfs/layouts/20...,07/2024,2024/07.pdf,"ERROR: 503, message='Service Unavailable', url...",False,None,None,None,None,None
16,17,17/2024,Sholavaram PU,Nayar II,https://www.cmdachennai.gov.in/pdfs/layouts/20...,17/2024,2024/17.pdf,"ERROR: 503, message='Service Unavailable', url...",False,None,None,None,None,None
36,37,37/2024,St.Thomas Mount PU,Kaspapuram,https://www.cmdachennai.gov.in/pdfs/layouts/20...,37/2024,2024/37.pdf,"ERROR: 503, message='Service Unavailable', url...",False,None,None,None,None,None
69,70,70/2024,Kundrathur PU,Kulathuvancheri,https://www.cmdachennai.gov.in/pdfs/layouts/20...,70/2024,2024/70.pdf,"ERROR: 503, message='Service Unavailable', url...",False,None,None,None,None,None
80,81,81/2024,Kundrathur PU,Kovur,https://www.cmdachennai.gov.in/pdfs/layouts/20...,81/2024,2024/81.pdf,"ERROR: 503, message='Service Unavailable', url...",False,None,None,None,None,None
93,94,94/2024,Poonamallee PU,"Sithukkadu,",https://www.cmdachennai.gov.in/pdfs/layouts/20...,94/2024,2024/94.pdf,"ERROR: 503, message='Service Unavailable', url...",False,None,None,None,None,None
94,95,95/2024,St.Thomas Mount PU,Agaramthen,https://www.cmdachennai.gov.in/pdfs/layouts/20...,95/2024,2024/95.pdf,"ERROR: 503, message='Service Unavailable', url...",False,None,None,None,None,None
127,128,128/2024,St.Thomas Mount PU,Vengambakkam,https://www.cmdachennai.gov.in/pdfs/layouts/20...,128/2024,2024/128.pdf,"ERROR: 503, message='Service Unavailable', url...",False,None,None,None,None,None
129,130,130/2024,Puzhal PU,Vilangadupakkam,https://www.cmdachennai.gov.in/pdfs/layouts/20...,130/2024,2024/130.pdf,"ERROR: 503, message='Service Unavailable', url...",False,None,None,None,None,None
130,131,131/2024,St.Thomas Mount PU,Arasankazhani,https://www.cmdachennai.gov.in/pdfs/layouts/20...,131/2024,2024/131.pdf,"ERROR: 503, message='Service Unavailable', url...",False,None,None,None,None,None
